In [3]:
!pip install -q torch torchvision h5py huggingface_hub tqdm


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [8]:
!pip install -q torchmetrics


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install -U albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 kB 6.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: albumentations
    Found existing installation: albumentations 2.0.6
    Uninstalling albumentations-2.0.6:
      Successfully uninstalled albumentations-2.0.6

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [9]:
# Thư viện
import os
import sys
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler
from torchvision import transforms, models
from torch.cuda.amp import GradScaler, autocast
from torchmetrics.classification import MulticlassAccuracy
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
# Đường dẫn
INPUT_DIR = "/kaggle/input/galaxy10-decals"         # thay bằng tên dataset của bạn
H5_PATH   = f"{INPUT_DIR}/Galaxy10_DECals.h5"
SAVE_PATH = "/kaggle/working/galaxy10_b0.pth"       # checkpoint & đồ thị sẽ lưu ở đây
assert os.path.exists(H5_PATH), f"❌ Không tìm thấy file HDF5: {H5_PATH}"
print("✅ Đã tìm thấy:", H5_PATH)

/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


✅ Đã tìm thấy: /kaggle/input/galaxy10-decals/Galaxy10_DECals.h5


In [10]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torchvision.transforms as T
IMG_SIZE = 224
MEAN, STD = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
# Albumentations chịu trách nhiệm về resize/augmentation chính:
alb_train = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.8, 1.0), ratio=(0.75, 1.333)),
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=180, p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.GaussianBlur(p=0.2),
    A.Normalize(MEAN, STD),
    ToTensorV2(),   # kết quả trả về là Tensor (C, H, W)
])
alb_val = A.Compose([
    A.Resize(height=IMG_SIZE, width=IMG_SIZE),
    A.Normalize(MEAN, STD),
    ToTensorV2(),
])
torch_train_extra = T.RandomErasing(
    p=0.3,
    scale=(0.02, 0.2),
    ratio=(0.3, 3.3),
    value=0
)
torch_val_extra = T.Lambda(lambda x: x)

In [11]:
# Dataset 
class Galaxy10H5(Dataset):
    def __init__(self, h5_path, indices=None, transform=None, extra_tfm=None):
        self.h5 = h5py.File(h5_path, "r")
        self.imgs = self.h5["images"]
        self.labs = self.h5["ans"]
        self.idxs = np.arange(len(self.imgs)) if indices is None else indices
        self.transform = transform
        self.extra = extra_tfm
    def __len__(self):
        return len(self.idxs)
    def __getitem__(self, i):
        # Lấy ảnh + label tại vị trí self.idxs[i]
        img = self.imgs[self.idxs[i]]           # numpy array (256, 256, 3) uint8
        lab = int(self.labs[self.idxs[i]])      # label (0–9)
        if self.transform is not None:
            # Albumentations: input là np.ndarray, output là dict {'image': tensor}
            img = self.transform(image=img)["image"]  # bây giờ img là Tensor (C, H, W)
        if self.extra is not None:
            img = self.extra(img)  # extra nhận Tensor, trả Tensor
        return img, lab

In [12]:
from torch.utils.data import random_split, WeightedRandomSampler, DataLoader
import os
# Tạo dataset gốc KHÔNG transform (chỉ để random_split)
full_ds_base = Galaxy10H5(H5_PATH, transform=None, extra_tfm=None)
# Lấy kích thước và tách chỉ số: 80% train, 10% val, 10% test
N = len(full_ds_base)
n_val  = int(0.10 * N)
n_test = int(0.10 * N)
n_train = N - n_val - n_test
g = torch.Generator().manual_seed(42)
# Kết quả của random_split là 3 object Subset, mỗi Subset giữ .indices
train_subset, val_subset, test_subset = random_split(full_ds_base, [n_train, n_val, n_test], generator=g)
train_idxs = train_subset.indices
val_idxs   = val_subset.indices
test_idxs  = test_subset.indices
# Tạo dataset riêng cho train/val/test, gán transform/extra_tfm tương ứng
train_ds = Galaxy10H5(
    H5_PATH,
    indices=train_idxs,
    transform=alb_train,       # Albumentations cho train
    extra_tfm=torch_train_extra  # RandomErasing chỉ dùng trên train
)
val_ds = Galaxy10H5(
    H5_PATH,
    indices=val_idxs,
    transform=alb_val,         # Albumentations cho validation
    extra_tfm=torch_val_extra  # torch_val_extra = lambda x: x (không làm gì)
)
test_ds = Galaxy10H5(
    H5_PATH,
    indices=test_idxs,
    transform=alb_val,         # Xài đúng val transform (không augmentation ngẫu nhiên)
    extra_tfm=torch_val_extra
)
# Tạo WeightedRandomSampler cho train (cân bằng lớp)
targets = np.array([full_ds_base.labs[i] for i in train_idxs])
class_counts = np.bincount(targets, minlength=10)   # 10 classes
class_weights = 1.0 / class_counts
sample_weights = class_weights[targets]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
BATCH = 64
NUM_WORKERS = min(8, os.cpu_count())
# DataLoader
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH,
    sampler=sampler,       # dùng sampler để cân bằng
    num_workers=NUM_WORKERS,
    pin_memory=True
)
val_loader = DataLoader(
    val_ds,
    batch_size=BATCH,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
test_loader = DataLoader(
    test_ds,
    batch_size=BATCH,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
print(f"Train {len(train_ds)} | Val {len(val_ds)} | Test {len(test_ds)}")

Train 14190 | Val 1773 | Test 1773


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 10
model = timm.create_model("efficientnet_b2", pretrained=True, num_classes=NUM_CLASSES)
model = model.to(device)
# Focal Loss (xử lý imbalance)
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
    def forward(self, logits, targets):
        ce = torch.nn.functional.cross_entropy(logits, targets, reduction="none")
        p_t = torch.exp(-ce)
        loss = ((1 - p_t) ** self.gamma * ce).mean()
        return loss
criterion = FocalLoss(gamma=2.0)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
# OneCycleLR: max_lr=3e-3, epochs=20, steps_per_epoch=len(train_loader)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=3e-3,
    steps_per_epoch=len(train_loader),
    epochs=10
)
scaler = GradScaler()
metric = MulticlassAccuracy(num_classes=NUM_CLASSES).to(device)
SAVE_PATH = "galaxy10_ema.pth"

/tmp/ipykernel_10/551400850.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
from copy import deepcopy
from tqdm.auto import tqdm         
import torch
best_val_acc = 0.0
patience_cnt = 0
EPOCHS, PATIENCE = 10, 5         
ema_state = deepcopy(model.state_dict())
history = {
    "train_loss": [], 
    "val_loss": [], 
    "train_acc": [], 
    "val_acc": []
}
def run_epoch(loader, train=True):
    if train:
        model.train()
    else:
        model.eval()
    torch.set_grad_enabled(train)
    total_loss = 0.0
    total_samples = 0
    metric.reset()
    loop = tqdm(loader, leave=False)
    for x, y in loop:
        # Chuyển data lên device (cuda hoặc cpu)
        x, y = x.to(device), y.to(device)
        if train:
            optimizer.zero_grad()   # reset gradients chỉ khi train
        # Mixed precision: autocast cho forward + loss
        with torch.cuda.amp.autocast():
            logits = model(x)             # output shape: (B, NUM_CLASSES)
            loss = criterion(logits, y)   # FocalLoss hoặc CrossEntropy, v.v.
        if train:
            scaler.scale(loss).backward()  # scale loss rồi backward
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()              # OneCycleLR điều chỉnh lr sau mỗi batch
        # Cộng dồn tổng loss, tính số samples
        batch_size = x.size(0)
        total_loss += loss.item() * batch_size
        total_samples += batch_size
        # Cập nhật accuracy metric (dùng logits.softmax vì MulticlassAccuracy cần probs)
        with torch.no_grad():
            probs = logits.softmax(dim=1)
            metric.update(probs, y)
        # Hiển thị progress bar, show loss hiện tại
        loop.set_postfix(batch_loss=loss.item())
    # Kết thúc loop, tính loss trung bình và acc trung bình
    avg_loss = total_loss / total_samples
    avg_acc  = metric.compute().item()
    return avg_loss, avg_acc
for epoch in range(1, EPOCHS + 1):
    # Chạy 1 epoch train
    tr_loss, tr_acc = run_epoch(train_loader, train=True)
    # Chạy 1 epoch validation (train=False tự động tắt gradient)
    val_loss, val_acc = run_epoch(val_loader, train=False)
    # Lưu lại lịch sử để plot learning curves sau
    history["train_loss"].append(tr_loss)
    history["val_loss"].append(val_loss)
    history["train_acc"].append(tr_acc)
    history["val_acc"].append(val_acc)
    # In ra màn hình tiến độ epoch
    # In ra learning rate hiện tại (lấy từ optimizer)
    current_lr = optimizer.param_groups[0]["lr"]
    print(
        f"Epoch {epoch:02d} | "
        f"Train Loss: {tr_loss:.4f}, Train Acc: {tr_acc:.3%} | "
        f"Val   Loss: {val_loss:.4f}, Val   Acc: {val_acc:.3%} | "
        f"LR: {current_lr:.2e}"
    )
    # Cập nhật EMA (alpha = 0.999)
    for key in ema_state:
        ema_state[key] = 0.999 * ema_state[key] + 0.001 * model.state_dict()[key]
    # Nếu validation accuracy được cải thiện thì reset patience, lưu checkpoint EMA
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        patience_cnt = 0
        torch.save({"model": ema_state, "acc": best_val_acc}, SAVE_PATH)
        print("Save new best EMA model")
    else:
        patience_cnt += 1
        if patience_cnt >= PATIENCE:
            print("Early stopping triggered")
            break
print(f"\nFinished training. Best Validation Acc: {best_val_acc:.3%}")

  0%|          | 0/222 [00:00<?, ?it/s]

In [ ]:
ckpt = torch.load(SAVE_PATH, map_location=device)
model.load_state_dict(ckpt["model"])
model.eval()
all_preds = []
all_labels = []
with torch.no_grad(), autocast():
    for x, y in tqdm(test_loader, leave=False):
        x, y = x.to(device), y.to(device)
        logits = model(x)
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = y.cpu().numpy()
        all_preds.append(preds)
        all_labels.append(labels)
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)
print("Test accuracy:", (all_preds == all_labels).mean() * 100, "%")
print(classification_report(all_labels, all_preds, target_names=[f"class_{i}" for i in range(NUM_CLASSES)]))
# Vẽ confusion matrix
cm = confusion_matrix(all_labels, all_preds, normalize="true")
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt=".2f", cmap="Blues",
            xticklabels=[f"c{i}" for i in range(NUM_CLASSES)],
            yticklabels=[f"c{i}" for i in range(NUM_CLASSES)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Vẽ Learning Curve
# Sau khi training kết thúc (hoặc early-stopped), history chứa list loss/acc từng epoch
epochs = range(1, len(history["train_loss"]) + 1)
plt.figure(figsize=(12, 5))
# Plot loss
plt.subplot(1, 2, 1)
plt.plot(epochs, history["train_loss"], label="Train Loss")
plt.plot(epochs, history["val_loss"],   label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Training and Validation Loss")
# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history["train_acc"], label="Train Acc")
plt.plot(epochs, history["val_acc"],   label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Training and Validation Accuracy")
plt.tight_layout()
plt.show()